In [ ]:
import dualcodec
import torchaudio
from IPython.display import Audio

In [ ]:
model_id = "12hz_v1"

In [ ]:
path = "output_checkpoints_3/dualcodec_baseline_12hz_16384_4096_8vq/checkpoint/epoch-0000_step-0165000_loss-73.862129-dualcodec_baseline_12hz_16384_4096_8vq"
dualcodec_model = dualcodec.get_model(model_id, path, name="model.safetensors")

In [ ]:
dualcodec_inference = dualcodec.Inference(dualcodec_model=dualcodec_model, device="cpu")

In [ ]:
audio, sr = torchaudio.load("audio_samples/tara.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()

Audio(audio.squeeze(0), rate=24000)

In [ ]:
semantic_codes, acoustic_codes = dualcodec_inference.encode(audio, n_quantizers=8)

In [ ]:
out_audio = dualcodec_inference.decode(semantic_codes, acoustic_codes)
Audio(out_audio.cpu().squeeze(0), rate=24000)

In [ ]:
dualcodec_model_base = dualcodec.get_model(model_id)

In [ ]:
dualcodec_inference_base = dualcodec.Inference(dualcodec_model=dualcodec_model_base, device="cpu")

In [ ]:
audio, sr = torchaudio.load("audio_samples/tara.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()

Audio(audio.squeeze(0), rate=24000)

In [ ]:
semantic_codes, acoustic_codes = dualcodec_inference_base.encode(audio, n_quantizers=8)

In [ ]:
out_audio = dualcodec_inference_base.decode(semantic_codes, acoustic_codes)
Audio(out_audio.cpu().squeeze(0), rate=24000)

### Check

In [ ]:
from datasets import load_dataset
import os

local_dir = "/mnt/disks/emilia/emilia_dataset/Emilia/EN"
tar_paths = [filename for filename in os.listdir(local_dir) if filename.endswith(".tar")]
language = "EN"
            
dataset = load_dataset(
    local_dir,
    data_files={language.lower(): tar_paths},
    split=language.lower(),
    num_proc=40,
    cache_dir="/mnt/disks/emilia/emilia_cache",
)